In [ ]:
"""
<순환신경망 종류>
 * 심플 순환신경망(Simple RNN) 단점
  - 긴 문장(단어의 갯수가 많은 경우)을 학습하기 어려움
  - 문장이 길 수록(단어의 갯수가 많을수록) 초반의 정보(단어를 의미함)는 점진적으로 희석(소멸)됨
  - 즉, 멀리 떨어져 있는 단어의 정보를 인식하는데 어려움이 있음
    (문장의 앞뒤 문맥(단어를 의미함) 데이터의 기억이 단기기억으로 저장됨)
  - 이러한 단점을 보완한 모델이 LSTM 임
 
 * 장기기억 순환신경망(Long Shot-Term Memory, LSTM)
  - Simple RNN의 단점을 보완한 모델
  - 단기기억을 오래 기억할 수 있도록 고안된 모델
  - 많은 이전 단어들의 정보를 기억해야 하기 때문에 훈련 속도가 느림(단점)
  - 시스템 저장 공간이 많이 필요함(단점)
 
 * 게이트웨이 반복 단위 순환신경망(Gated Recurrent Unit, GRU)
  - LSTM의 장기기억 순환신경망의 개념을 그대로 적용하고, 단점을 보완한 모델
  - 너무 오래된 기억은 서서히 소멸해 나가면서, 최근 기억을 지속적으로 유지하는 방식을 사용함
  - 성능은 LSTM과 동일 (속도를 빠르게 하기 위한 모델임)
 
 *** RNN 순환신경망 모델들은 주로, RMSprop 옵티마이저를 사용함
"""

### 라이브러리 정의

In [9]:
import tensorflow as tf
from tensorflow import keras
from keras.layers import Input, Dense, Conv2D, MaxPool2D, Dropout, SimpleRNN, Embedding, LSTM
from keras.models import Sequential
from keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import train_test_split

### 텍스트 길이 정규화 라이브러리
# - 텍스트의 길이가 긴 경우에는 자르고, 길이가 짧은 경우에는 채움
from tensorflow.keras.preprocessing.sequence import pad_sequences


tf.keras.utils.set_random_seed(42)

### 사용할 데이터셋
# - 영화 리뷰 감상평 데이터(긍정/부정)
from keras.datasets import imdb

In [2]:
### 데이터 수집
(train_input, train_target), (test_input, test_target) = imdb.load_data(num_words=500)

print(train_input.shape, train_target.shape)
print(test_input.shape, test_target.shape)

(25000,) (25000,)
(25000,) (25000,)


### 훈련 : 검증 = 8 : 2 로 분리하기

In [3]:
### 변수명 : train_input, train_target, val_input, val_target
train_input, val_input, train_target, val_target = train_test_split(
    train_input, train_target, test_size=0.2, random_state=42
)

print(train_input.shape, train_target.shape)
print(val_input.shape, val_target.shape)
print(test_input.shape, test_target.shape)

(20000,) (20000,)
(5000,) (5000,)
(25000,) (25000,)


In [4]:
### 모든 데이터의 문장 길이를 확인하여 가장 많이 분포하는 위치를 기준으로 통일시키는 처리를 진행

### 훈련 독립변수의 각 문장데이터들의 문장길이(단어의 갯수)를 확인하여,
# - 하나의 배열에 저장하기
# - 변수명 : lengths
print(len(train_input), len(train_input[0]))

lengths = np.array([ len(x) for x in train_input ])

print(lengths)
print(lengths.shape)

20000 259
[259 520 290 ... 300  70  77]
(20000,)


In [5]:
### 훈련 데이터 스케일링
train_seq = pad_sequences(train_input, maxlen=100)
print(train_seq.shape, train_target.shape)

### 검증 데이터 스케일링
val_seq = pad_sequences(val_input, maxlen=100)
print(val_seq.shape, val_target.shape)

(20000, 100) (20000,)
(5000, 100) (5000,)


### 데이터 스케일링 하기(단어 길이 표준화하기)

In [6]:
### 텍스트 제거 또는 채우기 속성 추가
"""
<제거 및 채우기 속성>
 * truncating : maxlen보다 클 때 자르기 속성
  - truncating = "pre" : 앞쪽 제거(기본값으로 사용됨, 생략가능)
  - truncating = "post" : 뒤쪽 제거
  
 * padding : maxlen보다 작을 때 채우기 속성
  - padding = "pre" : 앞쪽을 0으로 채우기(기본값으로 사용됨, 생략가능)
  - padding = "post" : 뒤쪽을 0으로 채우기
"""

### 훈련 데이터 스케일링
train_seq = pad_sequences(train_input, maxlen=100, truncating="post", padding="post")
print(train_seq.shape, train_target.shape)

### 검증 데이터 스케일링
val_seq = pad_sequences(val_input, maxlen=100, truncating="post", padding="post")
print(val_seq.shape, val_target.shape)


(20000, 100) (20000,)
(5000, 100) (5000,)


### LSTM 훈련 모델 생성하기

In [11]:
### 모델 생성하기
model = Sequential([
    # 입력계층 (입베딩 계층) 추가하기
    Embedding(
        input_dim = 500,
        output_dim = 16,
        input_length = 100
    ),
    # 은닉계층 (LSTM 계층) 추가하기 : RNN모델의 이름만 바꿔주면 됩니다.
    LSTM(
        units = 8
    ),
    
    # 출력계층 추가하기
    Dense(
        units = 1,
        activation = "sigmoid"
    )
])

In [12]:
model.summary()

"""
<Param 계산공식>
 * embedding = 말뭉치갯수 * 출력갯수 = 500 * 16 = 8,000
 
 * LSTM = 가중치 세트 갯수 * (출력갯수 * (입력갯수 + 출력갯수 + 1))
        = 4 * (8 * (16 + 8 + 1)) = 800
        
<가중치 세트 설명>
  - 총 4개의 세트로 구성되어 있음(이떄 세트를 -> 게이트라고 칭하기도 합니다.)
  
 * 셀상태관리
  - LSTM의 장기 기억을 담당
  - 셀 상태는 정보를 장기간 동안 유지할 수 있도록 관리됨
  - 입력/망각/출력 게이트를 통해서 셀 상태 정보가 업데이트 됩니다.
 
 * 입력게이트
  - 현재 시점의 입력 데이터가 셀 상태에 얼마나 반영될지를 결정
  - 새로운 정보를 얼마나 저장할지를 조절하는 역할을 수행
 
 * 망각게이트
  - 이전 셀 상태에서 얼마나 많은 정보를 잊어버릴지 결정
  - 이전 상태의 정보를 얼마나 제거할지를 조절하는 역할
 
 * 출력게이트
  - 셀 상태에서 어떤 정보를 출력할지 결정
  - 셀 상태를  현재 시점의 출력으로 변환하는 역할
 
 * 가중치 업데이트 : 위 4개의 가중치 세트가 동시에 작동하면서
                    출력에 가장 가까운 단어 조합을 만들어 냄...
"""

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 100, 16)           8000      
                                                                 
 lstm (LSTM)                 (None, 8)                 800       
                                                                 
 dense (Dense)               (None, 1)                 9         
                                                                 
Total params: 8,809
Trainable params: 8,809
Non-trainable params: 0
_________________________________________________________________


'\n<Param 계산공식>\n * embedding = 말뭉치갯수 * 출력갯수 = 500 * 16 = 8,000\n \n * LSTM = 가중치 세트 갯수 * (출력갯수 * (입력갯수 + 출력갯수 + 1))\n        = 4 * (8 * (16 + 8 + 1)) = 800\n        \n<가중치 세트 설명>\n  - 총 4개의 세트로 구성되어 있음(이떄 세트를 -> 게이트라고 칭하기도 합니다.)\n  \n * 셀상태관리\n  - LSTM의 장기 기억을 담당\n  - 셀 상태는 정보를 장기간 동안 유지할 수 있도록 관리됨\n  - 입력/망각/출력 게이트를 통해서 셀 상태 정보가 업데이트 됩니다.\n \n * 입력게이트\n  - 현재 시점의 입력 데이터가 셀 상태에 얼마나 반영될지를 결정\n  - 새로운 정보를 얼마나 저장할지를 조절하는 역할을 수행\n \n * 망각게이트\n  - 이전 셀 상태에서 얼마나 많은 정보를 잊어버릴지 결정\n  - 이전 상태의 정보를 얼마나 제거할지를 조절하는 역할\n \n * 출력게이트\n  - 셀 상태에서 어떤 정보를 출력할지 결정\n  - 셀 상태를  현재 시점의 출력으로 변환하는 역할\n \n * 가중치 업데이트 : 위 4개의 가중치 세트가 동시에 작동하면서\n                    출력에 가장 가까운 단어 조합을 만들어 냄...\n'

### 훈련모델 설정하기(compile)

In [13]:
### RMSprop 사용, 학습율 0.0001 사용, 정확도 출력
model.compile(
    optimizer = keras.optimizers.RMSprop(learning_rate=0.0001),
    loss = "binary_crossentropy",
    metrics = ["accuracy"]
)

### 콜백함수 정의하기

In [14]:
mc = ModelCheckpoint(
    "./model/best_LSTM_model.keras",
    save_best_only = True
)

es = EarlyStopping(
    patience = 3,
    restore_best_weights = True
)

### 모델 훈련시키기

In [15]:
history = model.fit(train_seq,
                    train_target,
                    validation_data=(val_seq,
                                    val_target),
                    epochs=100,
                    batch_size=64,
                    callbacks=[mc, es]
                    )

Epoch 1/100
313/313 [==============================] - 8s 12ms/step - loss: 0.6928 - accuracy: 0.5150 - val_loss: 0.6926 - val_accuracy: 0.5142
Epoch 2/100
313/313 [==============================] - 3s 11ms/step - loss: 0.6922 - accuracy: 0.5307 - val_loss: 0.6920 - val_accuracy: 0.5222
Epoch 3/100
313/313 [==============================] - 3s 10ms/step - loss: 0.6912 - accuracy: 0.5473 - val_loss: 0.6909 - val_accuracy: 0.5318
Epoch 4/100
313/313 [==============================] - 3s 10ms/step - loss: 0.6895 - accuracy: 0.5587 - val_loss: 0.6884 - val_accuracy: 0.5730
Epoch 5/100
313/313 [==============================] - 3s 11ms/step - loss: 0.6837 - accuracy: 0.6039 - val_loss: 0.6749 - val_accuracy: 0.6356
Epoch 6/100
313/313 [==============================] - 3s 10ms/step - loss: 0.6426 - accuracy: 0.6736 - val_loss: 0.6140 - val_accuracy: 0.7008
Epoch 7/100
313/313 [==============================] - 3s 10ms/step - loss: 0.6008 - accuracy: 0.7078 - val_loss: 0.5935 - val_accuracy:

### 최적의 모델 읽어들여서 훈련 / 검증 / 테스트 성능 검증하기

In [20]:
test_seq = pad_sequences(test_input, maxlen=100, truncating="post", padding="post")
test_seq.shape

(25000, 100)

In [26]:
best_model = keras.models.load_model("./model/best_LSTM_model.keras")

train_score = best_model.evaluate(train_seq, train_target)
val_score = best_model.evaluate(val_seq, val_target)
test_score = best_model.evaluate(test_seq, test_target)

print(f"훈련 데이터   | 손실율 : {train_score[0]}, 정확도 : {train_score[1]}")
print(f"검증 데이터   | 손실율 : {val_score[0]}, 정확도 : {val_score[1]}")
print(f"테스트 데이터 | 손실율 : {test_score[0]}, 정확도 : {test_score[1]}")

782/782 [==============================] - 4s 5ms/step - loss: 0.4741 - accuracy: 0.7744
훈련 데이터   | 손실율 : 0.4524226486682892, 정확도 : 0.7881500124931335
검증 데이터   | 손실율 : 0.48544707894325256, 정확도 : 0.7635999917984009
테스트 데이터 | 손실율 : 0.4740878939628601, 정확도 : 0.7743600010871887


In [27]:
### 모델 생성하기
model = Sequential([
    # 입력계층 (입베딩 계층) 추가하기
    Embedding(
        input_dim = 500,
        output_dim = 16,
        input_length = 100
    ),
    # 은닉계층 (LSTM 계층) 추가하기 : RNN모델의 이름만 바꿔주면 됩니다.
    LSTM(
        units = 8
    ),
    Dense(
        units = 64,
        activation = "relu"
    ),
    Dense(
        units = 32,
        activation = "relu"
    ),
    # 출력계층 추가하기
    Dense(
        units = 1,
        activation = "sigmoid"
    )
])

In [29]:
model.compile(
    optimizer = keras.optimizers.Adam(learning_rate=0.001),
    loss = "binary_crossentropy",
    metrics = ["accuracy"]
)

In [30]:
mc = ModelCheckpoint(
    "./model/best_LSTM_model.keras",
    save_best_only = True
)

es = EarlyStopping(
    patience = 5,
    restore_best_weights = True
)

In [31]:
history = model.fit(train_seq,
                    train_target,
                    validation_data=(val_seq,
                                    val_target),
                    epochs=100,
                    batch_size=128,
                    callbacks=[mc, es]
                    )

Epoch 1/100
157/157 [==============================] - 3s 13ms/step - loss: 0.6749 - accuracy: 0.5621 - val_loss: 0.5738 - val_accuracy: 0.7036
Epoch 2/100
157/157 [==============================] - 2s 12ms/step - loss: 0.5173 - accuracy: 0.7509 - val_loss: 0.5145 - val_accuracy: 0.7502
Epoch 3/100
157/157 [==============================] - 2s 11ms/step - loss: 0.4795 - accuracy: 0.7746 - val_loss: 0.4900 - val_accuracy: 0.7610
Epoch 4/100
157/157 [==============================] - 2s 11ms/step - loss: 0.4707 - accuracy: 0.7754 - val_loss: 0.5043 - val_accuracy: 0.7548
Epoch 5/100
157/157 [==============================] - 2s 11ms/step - loss: 0.4638 - accuracy: 0.7796 - val_loss: 0.4827 - val_accuracy: 0.7630
Epoch 6/100
157/157 [==============================] - 2s 10ms/step - loss: 0.4587 - accuracy: 0.7783 - val_loss: 0.4827 - val_accuracy: 0.7640
Epoch 7/100
157/157 [==============================] - 2s 10ms/step - loss: 0.4566 - accuracy: 0.7800 - val_loss: 0.4812 - val_accuracy:

In [32]:
best_model = keras.models.load_model("./model/best_LSTM_model.keras")

train_score = best_model.evaluate(train_seq, train_target)
val_score = best_model.evaluate(val_seq, val_target)
test_score = best_model.evaluate(test_seq, test_target)

print(f"훈련 데이터   | 손실율 : {train_score[0]}, 정확도 : {train_score[1]}")
print(f"검증 데이터   | 손실율 : {val_score[0]}, 정확도 : {val_score[1]}")
print(f"테스트 데이터 | 손실율 : {test_score[0]}, 정확도 : {test_score[1]}")

782/782 [==============================] - 3s 4ms/step - loss: 0.4692 - accuracy: 0.7734
훈련 데이터   | 손실율 : 0.44511157274246216, 정확도 : 0.7863500118255615
검증 데이터   | 손실율 : 0.4811630845069885, 정확도 : 0.7638000249862671
테스트 데이터 | 손실율 : 0.4692428410053253, 정확도 : 0.7734400033950806
